<a href="https://colab.research.google.com/github/elviawu/mask-data/blob/main/Mask_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json

# 利用 requests 對 API 來源發送一個請求
url = 'https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json'
res = requests.get(url)

# 將請求回應的內容存成一個字串格式
d = res.text

# 將長得像 json 格式的字串解析成字典或列表
data = json.loads(d)

print(data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [2]:
# 填入欄位名稱
med_count = {}
for d in data['features']:
    county = d['properties']['county']
    if county == "":
      county = d['properties']['address'][:3]
    if county not in med_count:
         med_count[county] = 1
    else:
      med_count[county] += 1


print(med_count)
# {'台北市': 123, '新北市': 456 ...}

{'臺北市': 340, '高雄市': 424, '臺中市': 436, '臺南市': 280, '基隆市': 57, '新竹市': 39, '嘉義市': 67, '新北市': 507, '桃園市': 271, '新竹縣': 48, '宜蘭縣': 76, '苗栗縣': 57, '彰化縣': 181, '南投縣': 67, '雲林縣': 131, '嘉義縣': 84, '屏東縣': 140, '澎湖縣': 11, '花蓮縣': 47, '臺東縣': 23, '金門縣': 6, '連江縣': 1}


In [ ]:
import sqlite3
import datetime
import requests
import json

conn = sqlite3.connect('example.db')
c = conn.cursor()

# 新增且清空資料表
c.execute('''CREATE TABLE IF NOT EXISTS pharmacies
             (city text, counts text, createdAt datetime)''')
c.execute('''DELETE FROM pharmacies''')
conn.commit()

# 新增資料

url = 'https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json'
res = requests.get(url)
d = res.text
data = json.loads(d)

med = {}

for d in data['features']:
  city = d['properties']['county']
  if city == "":
    city = d['properties']['address'][:3]
  if city not in med:
    med[city] = 1
  else:
    med[city] += 1
for m in med.items():
  city = m[0]
  counts = m[1]
  t = datetime.datetime.now()
  print(f"INSERT INTO pharmacies VALUES ('{city}', {counts}, '{t}')")
  c.execute(f"INSERT INTO pharmacies VALUES ('{city}', {counts}, '{t}')")
  conn.commit()

# 查詢資料
c.execute("SELECT * FROM pharmacies")
print(c.fetchall())

conn.commit()
conn.close()